In [0]:
# base_data_dir = "/FileStore/data_spark_streaming"

In [0]:
# lines = (spark.read
#          .format("text")
#          .option("lineSep",".")
#          .load(f"{base_data_dir}/data/text")
#          )

In [0]:
# from pyspark.sql.functions import explode, split, trim, lower

# raw_words = lines.select(explode(split(lines.value," ")).alias("word"))

# quality_words = (raw_words.select(lower(trim(raw_words.word)).alias("word"))
#                         .where("word is not null")
#                         .where("word rlike '[a-z]'")
#                 )
# wordCounts = quality_words.groupBy("word").count()



In [0]:
# (wordCounts.write
#             .format("delta")
#             .mode("overwrite")
#             .saveAsTable("word_count_table")
# )

In [0]:
class batchWC():
    def __init__(self):
        self.base_data_dir = "/FileStore/data_spark_streaming"

    def getRawData(self):
        from pyspark.sql.functions import explode, split
        lines = (spark.read
                 .format("text")
                 .option("lineSep",".")
                 .load(f"{self.base_data_dir}/data/text")
                 )
        return lines.select(explode(split(lines.value," ")).alias("word"))
    
    def getQualityData(self, rawDF):
        from pyspark.sql.functions import trim, lower
        return ( rawDF.select(lower(trim(rawDF.word)).alias("word"))
                        .where("word is not null")
                        .where("word rlike '[a-z]'")
                )
    
    def getWordCount(self,qualityDF):
        return qualityDF.groupBy("word").count()
    
    def overwriteWordCount(self, wordCountDF):
        ( wordCountDF.write
                    .format("delta")
                    .mode("overwrite")
                    .saveAsTable("word_count_table")
        )
        # wordCountDF.createOrReplaceTempView("word_count_table")

    def wordCount(self):
        print(f"\tExecuting Word Count...", end="")
        rawDF = self.getRawData()
        qualityDF = self.getQualityData(rawDF)
        resultDF = self.getWordCount(qualityDF)
        self.overwriteWordCount(resultDF)
        # spark.sql("select * from word_count_table").show()
        print("Done")